In [1]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_datetime64_ns_dtype
import inflect

In [2]:
def populate_org_dict(target_df):
    result = {}

    for i in range(len(target_df)):
        org_id = target_df.loc[i, 'organization_id']
        org_id = int(org_id) if not pd.isnull(org_id) else 0

        if org_id in result:
            result[org_id] += ', ' + str(target_df.loc[i, '_id'])
        else:
            result[org_id] = str(target_df.loc[i, '_id'])
    return result

In [3]:
def generate_org_full_table(ticket_dict, user_dict):
    try:
        df_ticket_dict = pd.DataFrame(ticket_dict.items(), columns=['_id', 'tickets'])
        df_user_dict = pd.DataFrame(user_dict.items(), columns=['_id', 'users'])
        df_orgs_tmp = df_orgs.copy(deep=True)

        return pd.merge(pd.merge(df_orgs_tmp, df_ticket_dict, how='left', on='_id'), df_user_dict, how='left', on='_id')
    except BaseException as error:
        print('An exception occurred: {}'.format(error))
        return False

In [4]:
def generate_user_tkt_full_table(table_name, table_df, org_dict):
    new_column_name = 'users' if table_name == 'tickets' else 'tickets'
    table_df[new_column_name] = [list() for x in range(len(table_df.index))]                     
    
    for i in range(len(table_df)):
        organization_id = table_df.loc[i, 'organization_id']
        if organization_id in org_dict:
            table_df.at[i, new_column_name] = org_dict[organization_id].split(', ')
    
    return table_df

In [5]:
def get_column_type(target_df, column):
    if column in target_df.columns:
        if is_string_dtype(target_df[column]):
            return 'string'
        elif is_numeric_dtype(target_df[column]):
            return 'int'
        elif is_datetime64_ns_dtype(target_df[column]):
            return 'datetime'
        else:
            return 'unknown'
    else:
        return 'unknown'

In [6]:
# def print_formatted_result(result):
#     n_ordinal = inflect.engine()
    
#     if result.empty:
#         print('Search {0} for {1} with a value of {2}\nNo results found'.format(query_table, query_column, query_value[1]))
#     else:
#         columns = list(result.columns)
#         for i in range(len(result)):
#             for j in range(len(columns)):
#                 print('{0}: {1}\n'.format(columns[j], result.iloc[i, j]))
#             print('====== End of the %s result ======' % n_ordinal.ordinal(i+1))

In [7]:
def equal_query(target_df, column, column_type, value):
    global query_result
    
    if column_type == 'int':
        try:
            value = int(value)
        except ValueError:
            query_result = pd.DataFrame()
            return False

    query_result = target_df[target_df[column] == value]

In [8]:
def like_query(target_df, column, value):
    global query_result
    
    query_result = target_df[target_df[column].str.contains(value, na=False)]

In [9]:
def parse_query(input_2, input_3, input_4):
    CONST_TABLES_DICT = {1: ['users', 'users'], 2: ['tickets', 'tickets'], 3: ['orgs', 'organizations']}
    
    target_df = globals()['df_' + CONST_TABLES_DICT[input_2][0] + '_full']
    
    global query_table
    query_table = CONST_TABLES_DICT[input_2][1]
    
    global query_column
    query_column =  input_3
    
    global query_value
    query_value = input_4.split('@@')
    
    global query_result

    if query_column not in target_df.columns or query_value[1] == '':
        # Return empty result if the search column is invalide or the serach value is null
        query_result = pd.DataFrame()
        return False
    
    column_type = get_column_type(target_df, query_column)
    
    if query_value[0] == 'equal':
        equal_query(target_df, query_column, column_type, query_value[1])
    elif query_value[0] == 'like':
        if column_type == 'int':
            # Return empty result if try to search int column with like values
            query_result = pd.DataFrame()
            return False
        
        like_query(target_df, query_column, query_value[1])
    ## To-DO: Add more type of query functions, and also will support multiple columns query

In [10]:
df_tickets = pd.read_json('tickets.json')
df_users = pd.read_json('users.json')
df_orgs = pd.read_json('organizations.json')

org_to_tkt_dict = populate_org_dict(df_tickets)
df_users_full = generate_user_tkt_full_table('users', df_users.copy(deep=True), org_to_tkt_dict)

org_to_user_dict = populate_org_dict(df_users)
df_tickets_full = generate_user_tkt_full_table('tickets', df_tickets.copy(deep=True), org_to_user_dict)

df_orgs_full = generate_org_full_table(org_to_tkt_dict, org_to_user_dict)

query_table = None
query_column = None
query_value = None

In [11]:
def validate_result(expectations):
    validation_msg = ''
    if len(expectations) == 0:
        validation_msg = "\u2713 Passed" if query_result.empty else 'XXX Failed XXX'
    elif query_result.empty:
        validation_msg = "\u2713 Passed" if len(expectations) == 0 else 'XXX Failed XXX'
    else:
        result_ids = query_result['_id'].tolist()
        result_ids.sort()
        expectations.sort()
        validation_msg = "\u2713 Passed" if result_ids == expectations else 'XXX Failed XXX'
            
    print('Search {0} for {1} with a value of {2}: {3}'.format(query_table, query_column, query_value[1], validation_msg))

In [12]:
# Simple unit tests
parse_query(1, '_id', 'equal@@111')
validate_result([])
parse_query(1, '_ic', 'equal@@1')
validate_result([])
parse_query(1, 'role', 'equal@@admin')
validate_result([1, 2, 5, 7, 14, 17, 18, 21, 25, 28, 33, 37, 45, 46, 50, 54, 57, 60, 63, 64, 65, 66, 68, 74])
parse_query(1, '_id', 'equal@@')
validate_result([])
parse_query(1, '_id', 'like@@1')
validate_result([])
parse_query(2, '_id', 'like@@436bf9b0')
validate_result(['436bf9b0-1147-4c0a-8439-6f79833bff5b'])
parse_query(3, '_id', 'equal@@test')
validate_result([])
parse_query(3, '_id', 'equal@@-1')
validate_result([])
parse_query(3, '_id', 'equal@@106')
validate_result([106])
parse_query(3, '_id', 'equal@@  110  ')
validate_result([110])

Search users for _id with a value of 111: ✓ Passed
Search users for _ic with a value of 1: ✓ Passed
Search users for role with a value of admin: ✓ Passed
Search users for _id with a value of : ✓ Passed
Search users for _id with a value of 1: ✓ Passed
Search tickets for _id with a value of 436bf9b0: ✓ Passed
Search organizations for _id with a value of test: ✓ Passed
Search organizations for _id with a value of -1: ✓ Passed
Search organizations for _id with a value of 106: ✓ Passed
Search organizations for _id with a value of   110  : ✓ Passed
